In [1]:
def pla(w, x, y, step_size, passes, prediction):
    
    for c in range(passes):
        updated = False
        
        y_prediction = w[0]
        for i in range(len(x)):
            for j in range(len(x[i])):
                print("i is:", i, " and j is:", j)
                y_prediction += w[j+1]*x[i][j]
                prediction[i] = y_prediction

            if y_prediction * y[i] < 0:
                updated = True
                for j in range(len(w)):
                    print("new j is:", j)
                    if j > 0:
                        w[j] = w[j] + step_size * y[i] * x[i][j-1]
                    else:
                        w[j] = w[j] + step_size * y[i] * 1
        
        if not updated:
            break
            
    
                    

In [6]:
import random
import time
import pandas as pd

w,h = 2,50
points = [[0 for x in range(w)] for y in range(h)] 
cls = [0 for z in range(h)]

random.seed(time.time())

below_line = False
for i in range(50):
    points[i][0] = random.randint(1,100)
    points[i][1] = random.randint(1,100)
    
    #print(points[i][0] * points[i][1], " compared to ", (points[i][0] ** 2) + 1)
    
    while not below_line and (points[i][0] * points[i][1]) < ((points[i][0] ** 2) + 1) :
        if (random.randint(0,10) % 10):
            #print("adjust a top point at index:", i)
            points[i][0] = random.randint(1,100)
            points[i][1] = random.randint(1,100)
        else:
            #print("-----skipping adjustment------")
            break
        
    
    while below_line and (points[i][0] * points[i][1]) > ((points[i][0] ** 2) - 1):
        if (random.randint(0,10) % 10):
            #print("adjust a bottom point at index:", i)
            points[i][0] = random.randint(1,100)
            points[i][1] = random.randint(1,100)
        else:
            #print("-----skipping adjustment------")
            break
      
    if below_line :
        cls[i] = -1
    else:
        cls[i] = 1
    
    if i == 24:
        below_line = True 
        


prediction = [0 for z in range(h)]

#print(points)
#print(cls)

df = pd.DataFrame(points, cls, columns=("X", "Y"))
print(df)

#print(prediction)




     0   1
 1  23  26
 1   9  22
 1  32   7
 1  59  17
 1  59  61
 1  72   5
 1  72  92
 1  38  69
 1  42  56
 1  50  84
 1  53  72
 1  41  79
 1  51  89
 1  29  72
 1  33  77
 1  91  78
 1  63  94
 1  59  65
 1   2  18
 1  33  49
 1  65  81
 1  19  49
 1  50  53
 1   7  42
 1   6  54
-1   3  23
-1  85  47
-1  95  65
-1  73   7
-1  80  65
-1   3  92
-1  74  26
-1  61  48
-1  11   1
-1  66   9
-1  28   8
-1  63  43
-1  54  15
-1  69  16
-1  25  81
-1  77  55
-1   5  46
-1  53  18
-1  95  69
-1  65   3
-1  80  24
-1  59  18
-1   1  42
-1  34   9
-1  81  19


In [3]:
w = [1.0, 0.5, 0.5]
#type(w[0])

pla(w, points, cls, 1, 100, prediction)

i is: 0  and j is: 0
i is: 0  and j is: 1
i is: 1  and j is: 0
i is: 1  and j is: 1
i is: 2  and j is: 0
i is: 2  and j is: 1
i is: 3  and j is: 0
i is: 3  and j is: 1
i is: 4  and j is: 0
i is: 4  and j is: 1
i is: 5  and j is: 0
i is: 5  and j is: 1
i is: 6  and j is: 0
i is: 6  and j is: 1
i is: 7  and j is: 0
i is: 7  and j is: 1
i is: 8  and j is: 0
i is: 8  and j is: 1
i is: 9  and j is: 0
i is: 9  and j is: 1
i is: 10  and j is: 0
i is: 10  and j is: 1
i is: 11  and j is: 0
i is: 11  and j is: 1
i is: 12  and j is: 0
i is: 12  and j is: 1
i is: 13  and j is: 0
i is: 13  and j is: 1
i is: 14  and j is: 0
i is: 14  and j is: 1
i is: 15  and j is: 0
i is: 15  and j is: 1
i is: 16  and j is: 0
i is: 16  and j is: 1
i is: 17  and j is: 0
i is: 17  and j is: 1
i is: 18  and j is: 0
i is: 18  and j is: 1
i is: 19  and j is: 0
i is: 19  and j is: 1
i is: 20  and j is: 0
i is: 20  and j is: 1
i is: 21  and j is: 0
i is: 21  and j is: 1
i is: 22  and j is: 0
i is: 22  and j is: 1
i is: 23

i is: 40  and j is: 1
i is: 41  and j is: 0
i is: 41  and j is: 1
i is: 42  and j is: 0
i is: 42  and j is: 1
i is: 43  and j is: 0
i is: 43  and j is: 1
i is: 44  and j is: 0
i is: 44  and j is: 1
i is: 45  and j is: 0
i is: 45  and j is: 1
i is: 46  and j is: 0
i is: 46  and j is: 1
i is: 47  and j is: 0
i is: 47  and j is: 1
i is: 48  and j is: 0
i is: 48  and j is: 1
i is: 49  and j is: 0
i is: 49  and j is: 1
i is: 0  and j is: 0
i is: 0  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 1  and j is: 0
i is: 1  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 2  and j is: 0
i is: 2  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 3  and j is: 0
i is: 3  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 4  and j is: 0
i is: 4  and j is: 1
i is: 5  and j is: 0
i is: 5  and j is: 1
i is: 6  and j is: 0
i is: 6  and j is: 1
i is: 7  and j is: 0
i is: 7  and j is: 1
i is: 8  and j is: 0
i is: 8  and j is: 1
i is: 9  and j is: 0
i is: 9  and j is: 1
i is: 10  and j is

new j is: 2
i is: 15  and j is: 0
i is: 15  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 16  and j is: 0
i is: 16  and j is: 1
i is: 17  and j is: 0
i is: 17  and j is: 1
i is: 18  and j is: 0
i is: 18  and j is: 1
i is: 19  and j is: 0
i is: 19  and j is: 1
i is: 20  and j is: 0
i is: 20  and j is: 1
i is: 21  and j is: 0
i is: 21  and j is: 1
i is: 22  and j is: 0
i is: 22  and j is: 1
i is: 23  and j is: 0
i is: 23  and j is: 1
i is: 24  and j is: 0
i is: 24  and j is: 1
i is: 25  and j is: 0
i is: 25  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 26  and j is: 0
i is: 26  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 27  and j is: 0
i is: 27  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 28  and j is: 0
i is: 28  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 29  and j is: 0
i is: 29  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 30  and j is: 0
i is: 30  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 31  and j is: 0
i is: 31  

i is: 48  and j is: 0
i is: 48  and j is: 1
i is: 49  and j is: 0
i is: 49  and j is: 1
i is: 0  and j is: 0
i is: 0  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 1  and j is: 0
i is: 1  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 2  and j is: 0
i is: 2  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 3  and j is: 0
i is: 3  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 4  and j is: 0
i is: 4  and j is: 1
i is: 5  and j is: 0
i is: 5  and j is: 1
i is: 6  and j is: 0
i is: 6  and j is: 1
i is: 7  and j is: 0
i is: 7  and j is: 1
i is: 8  and j is: 0
i is: 8  and j is: 1
i is: 9  and j is: 0
i is: 9  and j is: 1
i is: 10  and j is: 0
i is: 10  and j is: 1
i is: 11  and j is: 0
i is: 11  and j is: 1
i is: 12  and j is: 0
i is: 12  and j is: 1
i is: 13  and j is: 0
i is: 13  and j is: 1
i is: 14  and j is: 0
i is: 14  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 15  and j is: 0
i is: 15  and j is: 1
i is: 16  and j is: 0
i is: 16  and j is: 1
i is

i is: 38  and j is: 1
i is: 39  and j is: 0
i is: 39  and j is: 1
i is: 40  and j is: 0
i is: 40  and j is: 1
i is: 41  and j is: 0
i is: 41  and j is: 1
i is: 42  and j is: 0
i is: 42  and j is: 1
i is: 43  and j is: 0
i is: 43  and j is: 1
i is: 44  and j is: 0
i is: 44  and j is: 1
i is: 45  and j is: 0
i is: 45  and j is: 1
i is: 46  and j is: 0
i is: 46  and j is: 1
i is: 47  and j is: 0
i is: 47  and j is: 1
i is: 48  and j is: 0
i is: 48  and j is: 1
i is: 49  and j is: 0
i is: 49  and j is: 1
i is: 0  and j is: 0
i is: 0  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 1  and j is: 0
i is: 1  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 2  and j is: 0
i is: 2  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 3  and j is: 0
i is: 3  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 4  and j is: 0
i is: 4  and j is: 1
i is: 5  and j is: 0
i is: 5  and j is: 1
i is: 6  and j is: 0
i is: 6  and j is: 1
i is: 7  and j is: 0
i is: 7  and j is: 1
i is: 8  and j

...

i is: 8  and j is: 1
i is: 9  and j is: 0
i is: 9  and j is: 1
i is: 10  and j is: 0
i is: 10  and j is: 1
i is: 11  and j is: 0
i is: 11  and j is: 1
i is: 12  and j is: 0
i is: 12  and j is: 1
i is: 13  and j is: 0
i is: 13  and j is: 1
i is: 14  and j is: 0
i is: 14  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 15  and j is: 0
i is: 15  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 16  and j is: 0
i is: 16  and j is: 1
i is: 17  and j is: 0
i is: 17  and j is: 1
i is: 18  and j is: 0
i is: 18  and j is: 1
i is: 19  and j is: 0
i is: 19  and j is: 1
i is: 20  and j is: 0
i is: 20  and j is: 1
i is: 21  and j is: 0
i is: 21  and j is: 1
i is: 22  and j is: 0
i is: 22  and j is: 1
i is: 23  and j is: 0
i is: 23  and j is: 1
i is: 24  and j is: 0
i is: 24  and j is: 1
i is: 25  and j is: 0
i is: 25  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 26  and j is: 0
i is: 26  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 27  and j is: 0
i is: 27  and j is: 1
n

i is: 7  and j is: 1
i is: 8  and j is: 0
i is: 8  and j is: 1
i is: 9  and j is: 0
i is: 9  and j is: 1
i is: 10  and j is: 0
i is: 10  and j is: 1
i is: 11  and j is: 0
i is: 11  and j is: 1
i is: 12  and j is: 0
i is: 12  and j is: 1
i is: 13  and j is: 0
i is: 13  and j is: 1
i is: 14  and j is: 0
i is: 14  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 15  and j is: 0
i is: 15  and j is: 1
i is: 16  and j is: 0
i is: 16  and j is: 1
i is: 17  and j is: 0
i is: 17  and j is: 1
i is: 18  and j is: 0
i is: 18  and j is: 1
i is: 19  and j is: 0
i is: 19  and j is: 1
i is: 20  and j is: 0
i is: 20  and j is: 1
i is: 21  and j is: 0
i is: 21  and j is: 1
i is: 22  and j is: 0
i is: 22  and j is: 1
i is: 23  and j is: 0
i is: 23  and j is: 1
i is: 24  and j is: 0
i is: 24  and j is: 1
i is: 25  and j is: 0
i is: 25  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 26  and j is: 0
i is: 26  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 27  and j is: 0
i is: 27  and j i

i is: 13  and j is: 0
i is: 13  and j is: 1
i is: 14  and j is: 0
i is: 14  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 15  and j is: 0
i is: 15  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 16  and j is: 0
i is: 16  and j is: 1
i is: 17  and j is: 0
i is: 17  and j is: 1
i is: 18  and j is: 0
i is: 18  and j is: 1
i is: 19  and j is: 0
i is: 19  and j is: 1
i is: 20  and j is: 0
i is: 20  and j is: 1
i is: 21  and j is: 0
i is: 21  and j is: 1
i is: 22  and j is: 0
i is: 22  and j is: 1
i is: 23  and j is: 0
i is: 23  and j is: 1
i is: 24  and j is: 0
i is: 24  and j is: 1
i is: 25  and j is: 0
i is: 25  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 26  and j is: 0
i is: 26  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 27  and j is: 0
i is: 27  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 28  and j is: 0
i is: 28  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 29  and j is: 0
i is: 29  and j is: 1
new j is: 0
new j is: 1
new j is: 2


new j is: 0
new j is: 1
new j is: 2
i is: 4  and j is: 0
i is: 4  and j is: 1
i is: 5  and j is: 0
i is: 5  and j is: 1
i is: 6  and j is: 0
i is: 6  and j is: 1
i is: 7  and j is: 0
i is: 7  and j is: 1
i is: 8  and j is: 0
i is: 8  and j is: 1
i is: 9  and j is: 0
i is: 9  and j is: 1
i is: 10  and j is: 0
i is: 10  and j is: 1
i is: 11  and j is: 0
i is: 11  and j is: 1
i is: 12  and j is: 0
i is: 12  and j is: 1
i is: 13  and j is: 0
i is: 13  and j is: 1
i is: 14  and j is: 0
i is: 14  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 15  and j is: 0
i is: 15  and j is: 1
i is: 16  and j is: 0
i is: 16  and j is: 1
i is: 17  and j is: 0
i is: 17  and j is: 1
i is: 18  and j is: 0
i is: 18  and j is: 1
i is: 19  and j is: 0
i is: 19  and j is: 1
i is: 20  and j is: 0
i is: 20  and j is: 1
i is: 21  and j is: 0
i is: 21  and j is: 1
i is: 22  and j is: 0
i is: 22  and j is: 1
i is: 23  and j is: 0
i is: 23  and j is: 1
i is: 24  and j is: 0
i is: 24  and j is: 1
i is: 25  and j 

new j is: 1
new j is: 2
i is: 3  and j is: 0
i is: 3  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 4  and j is: 0
i is: 4  and j is: 1
i is: 5  and j is: 0
i is: 5  and j is: 1
i is: 6  and j is: 0
i is: 6  and j is: 1
i is: 7  and j is: 0
i is: 7  and j is: 1
i is: 8  and j is: 0
i is: 8  and j is: 1
i is: 9  and j is: 0
i is: 9  and j is: 1
i is: 10  and j is: 0
i is: 10  and j is: 1
i is: 11  and j is: 0
i is: 11  and j is: 1
i is: 12  and j is: 0
i is: 12  and j is: 1
i is: 13  and j is: 0
i is: 13  and j is: 1
i is: 14  and j is: 0
i is: 14  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 15  and j is: 0
i is: 15  and j is: 1
new j is: 0
new j is: 1
new j is: 2
i is: 16  and j is: 0
i is: 16  and j is: 1
i is: 17  and j is: 0
i is: 17  and j is: 1
i is: 18  and j is: 0
i is: 18  and j is: 1
i is: 19  and j is: 0
i is: 19  and j is: 1
i is: 20  and j is: 0
i is: 20  and j is: 1
i is: 21  and j is: 0
i is: 21  and j is: 1
i is: 22  and j is: 0
i is: 22  and j is: 1
i 

In [4]:
print(prediction)

[36079.0, -67855.5, -127629.5, -20727.0, 232747.5, 867589.5, 789691.5, 1053883.5, 1329012.0, 705544.5, 896088.0, 889464.0, 767133.0, 69423.0, -373188.0, -39511.5, 326488.0, 625533.0, 1246050.5, 1134961.5, 1074903.0, 1497288.5, 1663822.5, 1850259.5, 1777151.0, 1436638.5, 1506026.5, 874321.5, 88680.0, -16478.0, -260581.5, -1050676.0, -1558001.5, -1143648.0, -1694402.0, -2459696.0, -2618299.0, -2773274.5, -2907293.5, -3287908.0, -3609771.0, -3066541.5, -3063833.0, -3160326.5, -3616319.0, -4350476.5, -4502743.5, -5113928.0, -5770273.0, -6364508.5]


In [5]:
print(cls)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


In [6]:
print(w)

[-26.0, -9963.5, 7418.5]
